In [15]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot


import datetime
from sklearn.linear_model import LinearRegression

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Workspace, Dataset
from azureml.core.experiment import Experiment
from azureml.core import Model

from datetime import datetime

from azureml.pipeline.steps import ParallelRunConfig, ParallelRunStep
from azureml.data import OutputFileDatasetConfig
from azureml.core.runconfig import DockerConfiguration
from azureml.core import Environment
from azureml.core.runconfig import DEFAULT_CPU_IMAGE


In [6]:
# get the workspace from config.json
ws = Workspace.from_config()
# get the datastore to upload our data
# datastore = ws.get_default_datastore()

In [19]:
capstone_env = 'pytown_energymonitor'

In [22]:
%%writefile ../$capstone_env/capstone_batch_environment.yml
# windows users
# %%writefile $experiment_folder\batch_environment.yml
name: batch_environment
dependencies:
- python=3.8
- numpy
- pandas
- scikit-learn
- pip:
  - azureml-core

Writing ../pytown_energymonitor/capstone_batch_environment.yml


In [27]:


# Create an Environment for the experiment
batch_env = Environment.from_conda_specification("pytown_energymonitor/capstone_batch_environment.yml")
batch_env.docker.base_image = DEFAULT_CPU_IMAGE
print('Configuration ready.')

TypeError: from_conda_specification() missing 1 required positional argument: 'file_path'

In [18]:
# # Get the batch dataset for input
batch_data_set = ws.datasets['daily_load_data1']

# Set the output location
default_ds = ws.get_default_datastore()
output_dir = OutputFileDatasetConfig(name='capstone_inferences')

# Define the parallel run step step configuration
parallel_run_config = ParallelRunConfig(
    source_directory=pytown_energymonitor,
    entry_script="score.py",
    mini_batch_size="5",
    error_threshold=10,
    output_action="append_row",
    environment=batch_env,
    compute_target=inference_cluster,
    node_count=2)

parallel_step_name = "batchscoring-" + datetime.now().strftime("%Y%m%d%H%M")

# Create the parallel run step
parallelrun_step = ParallelRunStep(
    name=parallel_step_name,
    parallel_run_config=parallel_run_config,
    inputs=[batch_data_set.as_named_input('daily_load_data1')],
    output=output_dir,
    arguments=[],
    allow_reuse=True
)

print('Steps defined')

NameError: name 'pytown_energymonitor' is not defined